In [75]:
import pip

def import_or_install(package):
    try:
        __import__(package)
    except ImportError:
        pip.main(['install', package])     

In [73]:
libraries = ["langchain_openai", "langchain_community", "langchain", "pymysql", "chromadb"];

In [77]:
for library in libraries:
    import_or_install(library)

In [1]:
db_user = ""
db_password = ""
db_host = ""
db_name = ""

###Building a basic NL2SQL model

In [13]:
from langchain_community.utilities.sql_database import SQLDatabase
# db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}",sample_rows_in_table_info=1,include_tables=['customers','orders'],custom_table_info={'customers':"customer"})
db = SQLDatabase.from_uri(f"postgresql://{db_user}:{db_password}@{db_host}/{db_name}")

In [15]:
print(db.dialect)
print(db.get_usable_table_names())
print(db.table_info)

postgresql
['codon_usage', 'gene_aliases', 'gene_annotations', 'gff_annotations', 'protein_data']

CREATE TABLE codon_usage (
	codon VARCHAR(10), 
	aa CHAR(1), 
	freq NUMERIC(10, 4), 
	abundance NUMERIC(10, 4)
)

/*
3 rows from codon_usage table:
codon	aa	freq	abundance
UAA	*	0.9100	0.6900
UGA	*	0.2700	0.2100
UAG	*	0.1300	0.1000
*/


CREATE TABLE gene_aliases (
	gene_id VARCHAR(255), 
	alias1 VARCHAR(255), 
	alias2 VARCHAR(255), 
	alias3 VARCHAR(255), 
	alias4 VARCHAR(255), 
	alias5 VARCHAR(255), 
	alias6 VARCHAR(255), 
	alias7 VARCHAR(255), 
	alias8 VARCHAR(255), 
	alias9 VARCHAR(255)
)

/*
3 rows from gene_aliases table:
gene_id	alias1	alias2	alias3	alias4	alias5	alias6	alias7	alias8	alias9
PF3D7_1314600	PF13_0083	None	None	None	None	None	None	None	None
PF3D7_0209400	PF02_0090	PFB0423c	PFB0425c	None	None	None	None	None	None
PF3D7_1142200	PF11_0434	None	None	None	None	None	None	None	None
*/


CREATE TABLE gene_annotations (
	source VARCHAR(50), 
	gene_id VARCHAR(255), 
	symbol VARCHAR

In [18]:
import os
os.environ["OPENAI_API_KEY"] = ""
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = ""

# Implementing multi Database Query Using Langchain

## Creating a simple prompt and executing it using langchain

#### Getting the prompt from chatgpt

In [19]:
from langchain.chains import create_sql_query_chain # This chain generates SQL queries for the given database.
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
generate_query = create_sql_query_chain(llm, db)
query = generate_query.invoke({"question": "What is the average hydrophobicity of UPI00000001D7"})
# "what is price of `1968 Ford Mustang`"
print(query)

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

SELECT "avg_hydrophobicity" 
FROM protein_data 
WHERE entry = 'UPI00000001D7';


#### Getting the value of the query through the QuerySQLDataBase tool

In [20]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool # Tool for querying a SQL database.
execute_query = QuerySQLDataBaseTool(db=db)
execute_query.invoke(query)

'[(-0.2945578231292518,)]'

### Combining the above two steps together

In [21]:
chain = generate_query | execute_query
chain.invoke({"question": "What is the average hydrophobicity of UPI00000001D7"})

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

'[(-0.2945578231292518,)]'

### Investigating the instruction passed to the LLM to generate the query

In [ ]:
chain.get_prompts()[0].pretty_print()

You are a MySQL expert. Given an input question, first create a syntactically correct MySQL query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per MySQL. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in backticks (`) to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use CURDATE() function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the S

### Formatting the output that is sent back to the user

In [22]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

rephrase_answer = answer_prompt | llm | StrOutputParser()

# generate_query = create_sql_query_chain(llm, db)

chain = (
    RunnablePassthrough.assign(query=generate_query).assign(
        result= itemgetter("query") | execute_query
    )
    | rephrase_answer
)

chain.invoke({"question": "What is the average hydrophobicity of UPI00000001D7"})

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

"The average hydrophobicity of the protein with the entry 'UPI00000001D7' is approximately -0.2946."

## Adding few-shot examples to enhance relevancy of queries

In [24]:
# https://api.python.langchain.com/en/latest/prompts/langchain_core.prompts.few_shot.FewShotChatMessagePromptTemplate.html

examples = [
    {
        "input": "Retrieve all gene aliases for gene_id 'PF3D7_1314600'.",
        "query": "SELECT * FROM gene_aliases WHERE gene_id = 'PF3D7_1314600';"
    },
    {
        "input": "Count the total number of unique genes listed in the gene_annotations table.",
        "query": "SELECT COUNT(DISTINCT gene_id) FROM gene_annotations;"
    },
    {
        "input": "Find the most frequently used codon in the codon_usage table.",
        "query": "SELECT codon, MAX(freq) FROM codon_usage GROUP BY codon ORDER BY MAX(freq) DESC LIMIT 1;"
    },
    {
        "input": "List all protein data entries first seen after 2010.",
        "query": "SELECT * FROM protein_data WHERE first_seen > '2010-01-01';"
    },
    {
        "input": "Get details of gene annotations where the symbol is 'VAR'.",
        "query": "SELECT * FROM gene_annotations WHERE symbol = 'VAR';"
    },
    {
        "input": "Show sequences from the protein_data table with an average hydrophobicity greater than 0.",
        "query": "SELECT sequence FROM protein_data WHERE avg_hydrophobicity > 0;"
    },
    {
        "input": "List all genes and their aliases from gene_aliases where alias9 is not NULL.",
        "query": "SELECT gene_id, alias1, alias2, alias3, alias4, alias5, alias6, alias7, alias8, alias9 FROM gene_aliases WHERE alias9 IS NOT NULL;"
    },
    {
        "input": "Find all GFF annotations related to pseudogenes.",
        "query": "SELECT * FROM gff_annotations WHERE type = 'pseudogene';"
    },
    {
        "input": "Retrieve all protein entries for Homo sapiens.",
        "query": "SELECT * FROM protein_data WHERE organisms LIKE '%Homo sapiens%';"
    },
    {
        "input": "Show details of the longest protein sequence.",
        "query": "SELECT MAX(length), entry FROM protein_data GROUP BY entry ORDER BY MAX(length) DESC LIMIT 1;"
    },
    {
        "input": "List gene annotations categorized under biological process (BP).",
        "query": "SELECT * FROM gene_annotations WHERE aspect = 'BP';"
    },
    {
        "input": "Find codon usage details where the abundance is less than 0.2.",
        "query": "SELECT * FROM codon_usage WHERE abundance < 0.2;"
    },
    {
        "input": "Retrieve annotations for genes first annotated by VEuPathDB.",
        "query": "SELECT * FROM gene_annotations WHERE source = 'VEuPathDB';"
    },
    {
        "input": "List all gene_ids and their corresponding protein names with multiple aliases.",
        "query": "SELECT ga.gene_id, pd.protein_names FROM gene_aliases ga JOIN protein_data pd ON ga.gene_id = pd.entry WHERE ga.alias2 IS NOT NULL;"
    },
    {
        "input": "Find proteins with no known pfam assigned.",
        "query": "SELECT entry, protein_names FROM protein_data WHERE pfam IS NULL;"
    },
    {
        "input": "List all entries from gff_annotations between start positions 500000 and 600000.",
        "query": "SELECT * FROM gff_annotations WHERE start BETWEEN 500000 AND 600000;"
    },
    {
        "input": "Show all protein data entries last seen before 2010 but first seen after 2000.",
        "query": "SELECT * FROM protein_data WHERE first_seen > '2000-01-01' AND last_seen < '2010-01-01';"
    },
    {
        "input": "Aggregate the average frequency of each amino acid in codon_usage.",
        "query": "SELECT aa, AVG(freq) AS average_frequency FROM codon_usage GROUP BY aa;"
    },
    {
        "input": "Find all annotations that have a gene_type of 'protein_coding' and are assigned by 'GeneDB'.",
        "query": "SELECT * FROM gene_annotations WHERE gene_type = 'protein_coding' AND assigned_by = 'GeneDB';"
    },
    {
        "input": "List every unique organism in the protein_data table along with the count of entries for each.",
        "query": "SELECT organisms, COUNT(*) FROM protein_data GROUP BY organisms;"
    },
    {
        "input": "List all gene_ids along with their corresponding protein names and the last time they were seen.",
        "query": "SELECT ga.gene_id, pd.protein_names, pd.last_seen FROM gene_aliases ga JOIN protein_data pd ON ga.gene_id = pd.entry;"
    },
    {
        "input": "Retrieve gene annotations and corresponding alias information for gene_id 'PF3D7_0100100'.",
        "query": "SELECT ga.gene_id, ga.alias1, ga.alias2, ga.alias3, ga.alias4, ga.alias5, ga.alias6, ga.alias7, ga.alias8, ga.alias9, ann.description FROM gene_aliases ga JOIN gene_annotations ann ON ga.gene_id = ann.gene_id WHERE ga.gene_id = 'PF3D7_0100100';"
    },
    {
        "input": "Find all gene annotations that have corresponding protein data, showing the gene's symbol, protein names, and sequence.",
        "query": "SELECT ann.symbol, pd.protein_names, pd.sequence FROM gene_annotations ann JOIN protein_data pd ON ann.gene_id = pd.entry;"
    },
    {
        "input": "List all genes that have a 'protein_coding' type in gene_annotations and their corresponding protein names and sequences from protein_data.",
        "query": "SELECT ann.gene_id, pd.protein_names, pd.sequence FROM gene_annotations ann JOIN protein_data pd ON ann.gene_id = pd.entry WHERE ann.gene_type = 'protein_coding';"
    },
    {
        "input": "Show gene_id, all aliases, and description for genes that are associated with the GO term 'GO:0020013'.",
        "query": "SELECT ga.gene_id, ga.alias1, ga.alias2, ga.alias3, ga.alias4, ga.alias5, ga.alias6, ga.alias7, ga.alias8, ga.alias9, ann.description FROM gene_aliases ga JOIN gene_annotations ann ON ga.gene_id = ann.gene_id WHERE ann.go_id = 'GO:0020013';"
    },
    {
        "input": "Retrieve detailed annotations for all genes annotated by 'GeneDB' along with their aliases and last seen protein data.",
        "query": "SELECT ann.gene_id, ann.description, ga.alias1, pd.last_seen FROM gene_annotations ann JOIN gene_aliases ga ON ann.gene_id = ga.gene_id JOIN protein_data pd ON ann.gene_id = pd.entry WHERE ann.assigned_by = 'GeneDB';"
    },
    {
        "input": "List all GFF annotations and corresponding gene annotations for the sequences starting from position 500000 to 600000.",
        "query": "SELECT gff.seqid, gff.start, gff.stop, gff.type, ann.gene_id, ann.description FROM gff_annotations gff JOIN gene_annotations ann ON gff.seqid = ann.gene_id WHERE gff.start BETWEEN 500000 AND 600000;"
    },
    {
        "input": "Find proteins and their codon usage statistics where the protein was first seen before 2005 and the codon frequency is greater than 0.5.",
        "query": "SELECT pd.protein_names, cu.codon, cu.freq FROM protein_data pd JOIN codon_usage cu ON pd.entry = cu.codon WHERE pd.first_seen < '2005-01-01' AND cu.freq > 0.5;"
    },
    {
        "input": "Show all protein sequences and their gene annotations where the protein is related to Homo sapiens and has a hydrophobicity less than 0.",
        "query": "SELECT pd.sequence, ann.gene_id, ann.description FROM protein_data pd JOIN gene_annotations ann ON pd.entry = ann.gene_id WHERE pd.organisms LIKE '%Homo sapiens%' AND pd.avg_hydrophobicity < 0;"
    },
    {
        "input": "List all gene aliases and corresponding protein data for genes that have more than five aliases.",
        "query": "SELECT ga.gene_id, ga.alias1, ga.alias2, ga.alias3, ga.alias4, ga.alias5, ga.alias6, ga.alias7, ga.alias8, ga.alias9, pd.protein_names FROM gene_aliases ga JOIN protein_data pd ON ga.gene_id = pd.entry WHERE ga.alias6 IS NOT NULL;"
    }    
]


In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder,FewShotChatMessagePromptTemplate,PromptTemplate

# Whenever we pass in an example from the few shot prompts list, format it in this way
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "{input}\nSQLQuery:"),
        ("ai", "{query}"),
    ]
)

# This few shot prompt is just like we pass a few examples to the LLM as support before we let it answer any question
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
    input_variables=["input"], # Which variable holds the value sent from the user 
)
# print(few_shot_prompt.format())
# Running the print statement will show the following in the console 

# Human: Retrieve all gene aliases for gene_id 'PF3D7_1314600'.
# SQLQuery:
# AI: SELECT * FROM gene_aliases WHERE gene_id = 'PF3D7_1314600';
# Human: Count the total number of unique genes listed in the gene_annotations table.
# SQLQuery:
# AI: SELECT COUNT(DISTINCT gene_id) FROM gene_annotations;
# Human: Find the most frequently used codon in the codon_usage table.
# SQLQuery:
# AI: SELECT codon, MAX(freq) FROM codon_usage GROUP BY codon ORDER BY MAX(freq) DESC LIMIT 1;
# .......
# ...
# ...
# ..


### Dynamic few-shot example selection

#### Here we want to select relevant examples from the list of available examples

This uses semantic similarity which is an inbuilt feature in langchain to pick the most similar examples
As users ask queries and keep using the app, we can get more examples so the system can learn from it's users
Store the user interaction data in a kv store for fast retrieval

In [87]:
from langchain_community.vectorstores import Chroma
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_openai import OpenAIEmbeddings

vectorstore = Chroma()
vectorstore.delete_collection()
example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples, # Passing in all the available examples
    OpenAIEmbeddings(), # Using a pre trained embedding for both the query and the examples
    vectorstore, # Vector database that helps us with semantic search
    k=2, # Pick just the top 2 examples that are semantically similar to the input question from the user
    input_keys=["input"], # Helps us pick the input queries that were similar to the previously asked queries, in real life, as users ask queries and keep using the app, we can get more examples so the system can learn from it's users
)
example_selector.select_examples({"input": "What is the average hydrophobicity of UPI00000001D7"})

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

[{'input': 'Show sequences from the protein_data table with an average hydrophobicity greater than 0.',
  'query': 'SELECT sequence FROM protein_data WHERE avg_hydrophobicity > 0;'},
 {'input': 'Show all protein sequences and their gene annotations where the protein is related to Homo sapiens and has a hydrophobicity less than 0.',
  'query': "SELECT pd.sequence, ann.gene_id, ann.description FROM protein_data pd JOIN gene_annotations ann ON pd.entry = ann.gene_id WHERE pd.organisms LIKE '%Homo sapiens%' AND pd.avg_hydrophobicity < 0;"}]

#### Using this example selector within our few shot prompt so we only pick relevant examples

In [88]:
few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    example_selector=example_selector, # Here rather than passing in all the examples, we pass in the example selector object 
    input_variables=["input","top_k"],
)
print(few_shot_prompt.format(input="What is the average hydrophobicity of UPI00000001D7"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

Human: Show sequences from the protein_data table with an average hydrophobicity greater than 0.
SQLQuery:
AI: SELECT sequence FROM protein_data WHERE avg_hydrophobicity > 0;
Human: Show all protein sequences and their gene annotations where the protein is related to Homo sapiens and has a hydrophobicity less than 0.
SQLQuery:
AI: SELECT pd.sequence, ann.gene_id, ann.description FROM protein_data pd JOIN gene_annotations ann ON pd.entry = ann.gene_id WHERE pd.organisms LIKE '%Homo sapiens%' AND pd.avg_hydrophobicity < 0;


### Building up to the final prompt to the LLM

In [89]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries."),
        few_shot_prompt,
        ("human", "{input}\n\nSQL: "),
    ]
)
print(final_prompt.format(input="What is the average hydrophobicity of UPI00000001D7",table_info="some table info"))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries.
Human: Show sequences from the protein_data table with an average hydrophobicity greater than 0.
SQLQuery:
AI: SELECT sequence FROM protein_data WHERE avg_hydrophobicity > 0;
Human: Show all protein sequences and their gene annotations where the protein is related to Homo sapiens and has a hydrophobicity less than 0.
SQLQuery:
AI: SELECT pd.sequence, ann.gene_id, ann.description FROM protein_data pd JOIN gene_annotations ann ON pd.entry = ann.gene_id WHERE pd.organisms LIKE '%Homo sapiens%' AND pd.avg_hydrophobicity < 0;
Human: What is the average hydrophobicity of UPI00000001D7

SQL: 


In [90]:
generate_query = create_sql_query_chain(llm, db, final_prompt) # Here along with the llm and the database information, we pass in the final prompt
chain = (
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)
chain.invoke({"question": "What is the average hydrophobicity of UPI00000001D7"})

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

"The average hydrophobicity of the protein with the entry 'UPI00000001D7' is -0.2945578231292518."

## Dynamic relevant table selection

### We additionally use information about the tables to pick and choose just the right tables for creating the queries. 

This also falls into the aspect of reducing token lengths for each query sent to the LLM

In [56]:
from operator import itemgetter
from langchain.chains.openai_tools import create_extraction_chain_pydantic
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
import pandas as pd

def get_table_details():
    # Read the CSV file into a DataFrame
    table_description = pd.read_csv("database_table_descriptions.tsv", sep="\t")
    table_docs = []

    # Iterate over the DataFrame rows to create Document objects
    table_details = ""
    for index, row in table_description.iterrows():
        table_details = table_details + "Table Name:" + row['Table'] + "\n" + "Table Description:" + row['Description'] + "\n\n"

    return table_details


class Table(BaseModel):
    """Table in SQL database."""

    name: str = Field(description="Name of table in SQL database.")

# table_names = "\n".join(db.get_usable_table_names())
table_details = get_table_details()
print(table_details)

Table Name:codon_usage
Table Description:This table stores data about the usage frequency and abundance of each codon. Codons are sequences of three DNA or RNA nucleotides that correspond to a specific amino acid or stop signal during protein synthesis. Each row represents a codon, its corresponding amino acid, and its frequency and abundance metrics in the genome.

Table Name:gene_aliases
Table Description:This table contains alias names for gene IDs. It helps in linking various names or identifiers that a single gene might be known by in different studies or databases. Each row contains a gene ID followed by multiple columns for potential aliases, helping to consolidate gene identification across different scientific literature or databases.

Table Name:gene_annotations
Table Description:The gene_annotations table stores detailed annotations for genes. This includes the source of the annotation, symbols, Gene Ontology IDs, references, evidence types, aspects (like biological process,

### Testing out the table selection chain

In [65]:
table_details_prompt = f"""Return the names of ALL the SQL tables that MIGHT be relevant to the user question. \
The tables are:

{table_details}

Remember to include ALL POTENTIALLY RELEVANT tables, even if you're not sure that they're needed."""

table_chain = create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt)
tables = table_chain.invoke({"input": "What is the average hydrophobicity of UPI00000001D7"})
tables

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

[Table(name='protein_data'), Table(name='gene_aliases')]

In [66]:
def get_tables(tables: List[Table]) -> List[str]:
    tables  = [table.name for table in tables]
    return tables

select_table = {"input": itemgetter("question")} | create_extraction_chain_pydantic(Table, llm, system_message=table_details_prompt) | get_tables
select_table.invoke({"question": "What is the average hydrophobicity of UPI00000001D7"})

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

['protein_data', 'gene_annotations']

In [91]:
# table_names_to_use is a default value that is present in the generateQuery chain 
# generate_query = create_sql_query_chain(llm, db)
# So in this chain, before we call the generate query, we begin by assigning a value to the `table_names_to_use` variable

chain = (
    RunnablePassthrough.assign(table_names_to_use=select_table) |
    RunnablePassthrough.assign(query=generate_query).assign(
        result=itemgetter("query") | execute_query
    )
    | rephrase_answer
)
chain.invoke({"question": "What is the average hydrophobicity of UPI00000001D7"})

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

"The average hydrophobicity of the protein with the entry 'UPI00000001D7' is -0.2945578231292518."

## Checking if the context is retained 

In [93]:
chain.invoke({"question": "Can you list their names?"})

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

"The next entry in the protein data table for organisms containing 'Homo sapiens' is for the protein with the ID UPI0000000980, which corresponds to Saccharomyces cerevisiae (Baker's yeast)."

On checking the trace of the above run, we observe that the following query has been created an executed 
```sql
SQL Query: SELECT ga.gene_id, ga.alias1, ga.alias2, ga.alias3, ga.alias4, ga.alias5, ga.alias6, ga.alias7, ga.alias8, ga.alias9 FROM gene_aliases ga;
```

This has returned a pretty long output answer, that fails the number of tokens that can be passed into the model.

## Adding memory to the chatbot so that it answers follow-up questions related to the database.






In [96]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.\n\nHere is the relevant table info: {table_info}\n\nBelow are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="messages"),  # We add another field here that holds context of the conversation between the chatbot and the user
        ("human", "{input}"),
    ]
)
print(final_prompt.format(input="What is the average hydrophobicity of UPI00000001D7",table_info="some table info",messages=[]))

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

System: You are a MySQL expert. Given an input question, create a syntactically correct MySQL query to run. Unless otherwise specificed.

Here is the relevant table info: some table info

Below are a number of examples of questions and their corresponding SQL queries. Those examples are just for referecne and hsould be considered while answering follow up questions
Human: Show sequences from the protein_data table with an average hydrophobicity greater than 0.
SQLQuery:
AI: SELECT sequence FROM protein_data WHERE avg_hydrophobicity > 0;
Human: Show all protein sequences and their gene annotations where the protein is related to Homo sapiens and has a hydrophobicity less than 0.
SQLQuery:
AI: SELECT pd.sequence, ann.gene_id, ann.description FROM protein_data pd JOIN gene_annotations ann ON pd.entry = ann.gene_id WHERE pd.organisms LIKE '%Homo sapiens%' AND pd.avg_hydrophobicity < 0;
Human: What is the average hydrophobicity of UPI00000001D7


In [97]:
from langchain.memory import ChatMessageHistory
history = ChatMessageHistory()

generate_query = create_sql_query_chain(llm, db,final_prompt)

chain = (
RunnablePassthrough.assign(table_names_to_use=select_table) |
RunnablePassthrough.assign(query=generate_query).assign(
    result=itemgetter("query") | execute_query
)
| rephrase_answer
)


In [98]:
question = "What is the average hydrophobicity of UPI00000001D7"
response = chain.invoke({"question": question,"messages":history.messages})
response

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

"The average hydrophobicity of the protein with the entry 'UPI00000001D7' is approximately -0.2946."

In [99]:
history.add_user_message(question)
history.add_ai_message(response)


In [100]:
history.messages

[HumanMessage(content='What is the average hydrophobicity of UPI00000001D7'),
 AIMessage(content="The average hydrophobicity of the protein with the entry 'UPI00000001D7' is approximately -0.2946.")]

In [101]:
response = chain.invoke({"question": "What about the next entry ?","messages":history.messages})
response

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

"The average hydrophobicity for the entry 'UPI0000000336' is -0.9732240437158476."

In [102]:
response = chain.invoke({"question": "What is the next entry in the database after UPI00000001D7 ?","messages":history.messages})
response

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"

'The next entry in the database after UPI00000001D7 is UPI0000000336.'

In [103]:
history.add_user_message("What is the next entry in the database after UPI00000001D7 ?")
history.add_ai_message(response)

In [105]:
history.messages

[HumanMessage(content='What is the average hydrophobicity of UPI00000001D7'),
 AIMessage(content="The average hydrophobicity of the protein with the entry 'UPI00000001D7' is approximately -0.2946."),
 HumanMessage(content='What is the next entry in the database after UPI00000001D7 ?'),
 AIMessage(content='The next entry in the database after UPI00000001D7 is UPI0000000336.')]